In [ ]:
%%bash
# install and update required packages
python3 -m pip install --upgrade pip -q
python3 -m pip install -r requirements.txt -q

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
''' W&B env. variables '''
import os
os.environ["WANDB_PROJECT"] = "report-experiments"
os.environ["WANDB_ENTITY"] = "dl4aed"
os.environ["WANDB_API_KEY"] = "***"
os.environ["WANDB_MODE"] = "dryrun" #or "run"
os.environ["WANDB_NOTEBOOK_NAME"] = "Training"

In [ ]:
''' Some imports '''
import tensorflow as tf
assert tf.__version__ >= "2.4.0"
import numpy as np
import pandas as pd
%matplotlib inline

In [ ]:
''' Allow typical dynamic GPU memory allocation'''
from utils import load_config, save_config, allow_growth, update
allow_growth()

In [ ]:
''' Read config, parsing CLI arguments and overwriting config '''
from argument_parser import parse_arguments

# read the config file
# it includes more or less all hyperparameter used in the model and preprocessing/training step
config = load_config(verbose=0)

# read args
parsed_arguments = parse_arguments()

# overwrite config
config = update(config, parsed_arguments)

In [ ]:
from wandb_utils import WandbWrapper
# sync config with W&B
wandb_wrapper = WandbWrapper(config)
config = wandb_wrapper.get_config()

In [ ]:
# extract parameter classes
data_parameter = config["data_parameter"]
model_parameter = config["model_parameter"]
training_parameter = config["training_parameter"]

In [ ]:
''' Some small amount of reproducibility '''
tf.random.set_seed(training_parameter["seed"])
np.random.seed(training_parameter["seed"])

In [ ]:
''' Create model and save config '''
from network import Network
network = Network(model_parameter, training_parameter)
network.compile()
network.save()
# save config into model path
save_config(data_parameter, model_parameter, training_parameter, network)

In [ ]:
''' Data pipeline procedure '''
from preprocessor import Preprocessor
prep = Preprocessor(config=data_parameter)
prep.create_logger()
prep.load_mels()
prep.launch_trainval_pipeline(mode="train")
prep.launch_trainval_pipeline(mode="val")
prep.launch_test_pipeline(mode="fma")
prep.launch_test_pipeline(mode="test")

In [ ]:
''' Training procedure '''
from callbacks import *
additional_callbacks = []
additional_callbacks += [CreateEmbedding(network, training_parameter["embedding_period"], prep.test_ds, model_parameter["num_classes"])]
additional_callbacks += [ReconstructImages(network, training_parameter["reconstruction_period"], prep.test_ds, wandb_wrapper)]
additional_callbacks += [wandb_wrapper.get_callback(save_model=False)]

history = network.fit(prep.train_ds, validation_data=prep.val_ds, epochs=training_parameter["epochs"],
                   initial_epoch=network.epoch, callbacks=network.callbacks + additional_callbacks)

pd.DataFrame.from_dict(history.history).plot()

In [ ]:
''' Evaluation procedure '''
for mode in ["test", "fma"]:
    print(f"Creating embedding for {mode}")
    ce = CreateEmbedding(network, None, prep.datasets[mode], model_parameter["num_classes"])
    embeddings, labels = ce.create_embedding()
    print("Saving embedding as numpy file")
    np.save(f"{network.model_path}{mode}_embeddings.npy", embeddings, allow_pickle=True)
    np.save(f"{network.model_path}{mode}_labels.npy", labels, allow_pickle=True)